In [1]:
!pip install pymupdf

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### 1. 문서 로드(Load Documents)

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDFLoader 객체 정의
loader = PyMuPDFLoader("data/snow-white.pdf")

# 문서 로드
docs = loader.load()

print(f"문자의 페이지 수 : {len(docs)}")
print(docs[0].page_content)

문자의 페이지 수 : 6
백설공주
옛날어느왕국에공주님이태어났어요.
“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백
설공주라고불러야겠다.”
왕과왕비는갓태어난딸을보며기뻐했어요.
하지만기쁨도잠시, 왕비는곧세상을떠나고말았어
요.



In [4]:
# 메타데이터(__dict__)
print(docs[0].__dict__)

{'id': None, 'metadata': {'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, 'page_content': '백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.\n하지만기쁨도잠시, 왕비는곧세상을떠나고말았어\n요.\n', 'type': 'Document'}


### 2. 문서 분할(Split Documents)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 21


### 3. 임베딩(Embedding 생성)

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

### 4. DB 생성(벡터스토어 생성) 및 저장
* FAISS(Facebook AI Similarity Search)
    * 페이스북에서 개발한 유사도 검색 및 클러스터링 라이브러리
    * 벡터 데이터셋에서 빠른 유사도 검색을 수행해준다.

In [7]:
!pip install faiss-cpu

In [16]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [9]:
for doc in vectorstore.similarity_search("왕자"):
    print(doc.page_content)

왕자는깨어난백설공주를보고기뻐했어요.
“공주님, 나는이웃나라왕자입니다.”
“왕자님이나를다시살려주셨군요.”
“나와결혼해주시겠어요?”
“네, 좋아요!”
왕자는백설공주에게반해유리관을달라고부탁했어요.
일곱난쟁이는백설공주를잘지킨다는약속을받고유리관을
내주었지요.
그런데신하들이유리관을옮기다돌부리에툭! 백설공주목
에서사과조각이툭!
하지만기쁨도잠시, 왕비는곧세상을떠나고말았어
요.
북적이는소리에잠이깬백설공주는왕비를피해도망쳤다고
이야기했어요.
“불쌍한공주님, 우리와함께살아요. 조심조심또조심. 낯선
사람에게는문을열어주지마세요.”


### 5. 검색기(Retriever) 생성

In [10]:
# 벡터스토어에 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

In [11]:
retriever.invoke("백설공주와 일곱난쟁이는 어디서 만났어?")

[Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.'),
 Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 4, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='저녁이되자, 일곱난쟁이가돌아왔어요.\n난쟁이들은쓰러진백설공주를보고엉엉울었어요.\n백설공주는깊은잠에빠진것처럼보였지요.\n“백설공주님, 못된왕비의꾐에넘어갔군요.

### 6. 프롬프트 생성

<프롬프트 내용><br>
당신은 질문-답변 작업을 위한 어시스턴스 입니다.<br>
주어진 문맥을 사용하여 질문에 답변하세요.<br>
유치원 선생님이 아이이게 말하는 것처럼 매우 친절하고 부드러운 어조를 사용하세요.<br>
따뜻하고 친근한 방식으로 말하세요.<br>
답을 모르는 경우에는 모른다고 말씀하세요.<br>
한국어로 답변하세요.<br>

In [12]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    Use a very kind and gentle tone like a kindergarten teacher talking to a child.
    Speak in a warm and friendly way.
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

    #Context: 
    {context}

    #Question:
    {question}

    #Answer:
""")

### 7. LLM 모델 생성

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 8. Chain 생성

In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# chain 실행

question = "왕자는 어느 나라 출신이에요?"

response = chain.invoke(question)

print(response)

미안하지만, 왕자가 어느 나라 출신인지에 대한 정보는 여기 나와 있지 않아요.
